In [ ]:
import numpy as np
import tensorflow as tf
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array


def preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array, version=2)
        images.append(img_array)
    return np.array(images)

image_paths = ['path_to_image1.jpg', 'path_to_image2.jpg', ...]
images = preprocess_images(image_paths)

base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

model = Sequential([
    base_model,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')  
])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(class_names))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(class_names))


model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=10
)


y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')


feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

embeddings_test = feature_extractor.predict(X_test)
embeddings_train = feature_extractor.predict(X_train)

from sklearn.metrics.pairwise import cosine_similarity

scores = cosine_similarity(embeddings_test, embeddings_train)

closest_match = np.argmax(scores, axis=1)
accuracy = np.mean(closest_match == np.argmax(y_test, axis=1))
print(f'Biometric Identification Accuracy: {accuracy * 100:.2f}%')
